In [ ]:
import numpy as np
import h5py
import scipy
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage

# from utils import *
from nn_building_blocks import *

%matplotlib inline

### Dataset

In [ ]:
train_dataset = h5py.File('datasets/train_catvnoncat.h5')
train_x_orig = np.array(train_dataset["train_set_x"][:])
train_y = np.array(train_dataset["train_set_y"][:])
train_y = train_y.reshape(1, -1)

In [ ]:
test_dataset = h5py.File('datasets/test_catvnoncat.h5')
test_x_orig = np.array(test_dataset["test_set_x"][:])
test_y = np.array(test_dataset["test_set_y"][:])
test_y = test_y.reshape(1, -1)

In [ ]:
classes = np.array(test_dataset["list_classes"][:])

In [ ]:
# Example of a picture
index = 10
plt.imshow(train_x_orig[index])
plt.show()
print ("y = " + str(train_y[0,index]) + ". It's a " + classes[train_y[0,index]].decode("utf-8") +  " picture.")

In [ ]:
training_examples = train_x_orig.shape[0]
test_examples = test_x_orig.shape[0]
num_px = train_x_orig.shape[1]

In [ ]:
print ("Number of training examples: " + str(training_examples))
print ("Number of testing examples: " + str(test_examples))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

In [ ]:
train_x_flatten = train_x_orig.reshape(training_examples, -1)
test_x_flatten = test_x_orig.reshape(test_examples, -1)

In [ ]:
train_x = train_x_flatten / 255
test_x = test_x_flatten / 255

In [ ]:
xtr = train_x.T
xtt = test_x.T

In [ ]:
print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

### The 2 Layer Model

In [ ]:
n_x = 12288
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)
learning_rate = 0.0075

In [ ]:
def two_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    
    np.random.seed(1)
    grads = {}
    costs= []
    m = X.shape[1]

    (nx, nh, ny) = layers_dims
    
    parameters = initialize_parameters(nx, nh, ny)

    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    for i in range(num_iterations):
        A1, cache1 = linear_activation_forward(X, W1, b1, "relu")
        A2, cache2 = linear_activation_forward(A1, W2, b2, "sigmoid")

        cost = compute_cost(A2, Y)

        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))

        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, "sigmoid")
        dX, dW1, db1 = linear_activation_backward(dA1, cache1, "relu")

        grads["dW1"] = dW1
        grads["dW2"] = dW2
        grads["db1"] = db1
        grads["db2"] = db2

        parameters = update_parameters(parameters, grads, learning_rate)

        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]

        if print_cost and (i % 100 == 0 or i ==num_iterations-1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 ==0:
            costs.append(cost)

    return parameters, costs

In [ ]:
def plot_costs(costs, learning_rate=0.0075):
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

In [ ]:
parameters, costs = two_layer_model(xtr, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2, print_cost=False)

print("Cost after first iteration: " + str(costs[0]))

In [ ]:
parameters, costs = two_layer_model(xtr, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True)
plot_costs(costs, learning_rate)

In [ ]:
predictions_train = predict(xtr, train_y, parameters)

In [ ]:
predictions_test = predict(xtt, test_y, parameters)

### L Layered Neural Network

In [ ]:
layers_dims = [12288, 20, 7, 5, 1]

In [ ]:
def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    np.random.seed(1)
    costs = []

    parameters = initialize_parameters_deep_xavier(layers_dims)

    for i in range(num_iterations):
        AL, caches = L_model_forward(X, parameters)
        
        cost = compute_cost(AL, Y)
        
        grads = L_model_backward(AL, Y, caches)

        parameters = update_parameters(parameters, grads, learning_rate)

        if print_cost and (i%100 == 0 or i==num_iterations-1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i%100==0:
            costs.append(cost)
    
    return parameters, costs

In [ ]:
parameters, costs = L_layer_model(xtr, train_y, layers_dims, num_iterations = 1, print_cost = False)

print("Cost after first iteration: " + str(costs[0]))

In [ ]:
parameters, costs = L_layer_model(xtr, train_y, layers_dims, num_iterations = 2500, print_cost = True)
plot_costs(costs, learning_rate)

In [ ]:
pred_train = predict(xtr, train_y, parameters)

In [ ]:
pred_test = predict(xtt, test_y, parameters)

In [ ]:
## START CODE HERE ##
my_image = "goat-2775034_960_720-3704390797.jpg" # change this to the name of your image file 
my_label_y = [1] # the true class of your image (1 -> cat, 0 -> non-cat)
## END CODE HERE ##

fname = my_image
image = np.array(Image.open(fname).resize((num_px, num_px)))
plt.imshow(image)
plt.show()
image = image / 255.
image = image.reshape((1, num_px * num_px * 3)).T

my_predicted_image = predict(image, my_label_y, parameters)


print ("y = " + str(np.squeeze(my_predicted_image)) + ", your L-layer model predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")